In [1]:
import os
import pandas as pd
import torch
from PIL import Image
from transformers import ChineseCLIPProcessor, ChineseCLIPModel
import openpyxl
import warnings
from huggingface_hub.file_download import logger as hf_logger

d:\software\anaconda\envs\unet\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 禁用huggingface的symlinks警告
hf_logger.setLevel("ERROR")
# 或直接禁用所有UserWarning
warnings.filterwarnings("ignore", category=UserWarning)
# 初始化模型
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ChineseCLIPModel.from_pretrained("OFA-Sys/chinese-clip-vit-base-patch16").to(device)
processor = ChineseCLIPProcessor.from_pretrained("OFA-Sys/chinese-clip-vit-base-patch16")

In [5]:
# 路径设置
image_dir = r"D:\AAA\一\大论文\子研究二\缩略图"
text_excel_path = r"D:\AAA\一\大论文\子研究二\code\图文相似度.xlsx"
output_path = r"C:\Users\12150\OneDrive\Desktop\1.xlsx"

# 读取Excel文件
df = pd.read_excel(text_excel_path)
text_data = df[['id', 'title']].to_dict('records')

# 准备结果存储
results = []

# 处理每张图片
for item in text_data:
    img_id = str(item['id']).split('.')[0]  # 永远取第一个点之前的部分
    text = item['title']
    
    # 查找对应图片
    img_path = None
    for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
        temp_path = os.path.join(image_dir, f"{img_id}{ext}")
        if os.path.exists(temp_path):
            img_path = temp_path
            break
    
    if not img_path:
        print(f"未找到ID为 {img_id} 的图片")
        continue
    
    try:
        # 处理图片和文本
        image = Image.open(img_path)
        inputs = processor(text=[text], images=image, return_tensors="pt", padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # 计算相似度
        with torch.no_grad():
            outputs = model(**inputs)
            similarity = outputs.logits_per_image.item()  # 获取相似度分数
        
        results.append({
            'id': img_id,
            'title': text,
            'image_path': img_path,
            'similarity': similarity
        })
        
        print(f"处理完成: ID {img_id} - 相似度 {similarity:.4f}")
        
    except Exception as e:
        print(f"处理ID {img_id} 时出错: {str(e)}")
        continue

# 保存结果到Excel
if results:
    result_df = pd.DataFrame(results)
    result_df = result_df[['id', 'title', 'image_path', 'similarity']]  # 调整列顺序
    
    # 确保输出目录存在
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # 保存为Excel
    result_df.to_excel(output_path, index=False)
    print(f"结果已保存到: {output_path}")
else:
    print("没有可保存的结果")

处理完成: ID 1 - 相似度 45.9341
处理完成: ID 2 - 相似度 44.0987
处理完成: ID 3 - 相似度 48.1567
处理完成: ID 4 - 相似度 40.2982
处理完成: ID 5 - 相似度 45.8508
处理完成: ID 6 - 相似度 43.2125
处理完成: ID 7 - 相似度 44.9267
处理完成: ID 8 - 相似度 39.7825
处理完成: ID 9 - 相似度 45.0551
处理完成: ID 10 - 相似度 36.1317
处理完成: ID 11 - 相似度 48.3559
处理完成: ID 12 - 相似度 43.4803
处理完成: ID 13 - 相似度 44.5469
处理完成: ID 14 - 相似度 48.4359
处理完成: ID 15 - 相似度 42.4276
处理完成: ID 16 - 相似度 44.3894
处理完成: ID 17 - 相似度 44.7725
处理完成: ID 18 - 相似度 47.4460
处理完成: ID 19 - 相似度 49.7662
处理完成: ID 20 - 相似度 40.6561
处理完成: ID 21 - 相似度 41.1835
处理完成: ID 22 - 相似度 45.9119
处理完成: ID 23 - 相似度 47.9116
处理完成: ID 24 - 相似度 46.3806
处理完成: ID 25 - 相似度 47.8108
处理完成: ID 26 - 相似度 48.4824
处理完成: ID 27 - 相似度 38.6617
处理完成: ID 28 - 相似度 47.1570
处理完成: ID 29 - 相似度 46.1944
处理完成: ID 30 - 相似度 43.1825
处理完成: ID 31 - 相似度 44.5681
处理完成: ID 32 - 相似度 41.7253
处理完成: ID 33 - 相似度 45.4506
处理完成: ID 34 - 相似度 41.2540
处理完成: ID 35 - 相似度 43.9219
处理完成: ID 36 - 相似度 35.8882
处理完成: ID 37 - 相似度 45.1481
处理完成: ID 38 - 相似度 44.8487
处理完成: ID 39 - 相似度 42.